# Comparing Frequencies: Consensus Nomenclature
In this notebook, we exploit the GPCR [consensus nomenclature](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/mdciao.nomenclature.html) to compute and compare contact frequencies across four GPCRs that have very little sequence identity. 

Nevertheless, the consensus nomenclature will allow us to:

* Use the same function calls for all systems, regardless of the underlying primary sequence

* Compare the frequencies across systems by using consensus labels

The four systems we will be comparing are:         
* Beta 2 adrenergic receptor in complex with Gs-protein.  
  Provided kindly by Dr. H. Batebi
  
* Growth hormone secretagogue receptor type 1, ghrelin receptor for short.  
  Provided kindly by Dr. A. Vogel

* Neuropeptide Y receptor type 1, Y1 receptor for short, in apo form.  
  Provided kindly by Dr. A. Vogel.

* Active mu-opioid receptor bound to the agonist morphine.  
  Kindly made available for this purpose by the GPCRmd. 

All these example datasets will be downloaded on the fly using [mdciao.examples.fetch_example_data](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.examples.fetch_example_data.html#mdciao-examples-fetch-example-data), please note the individual references for each dataset there.

Also note that `mdciao` ships with all references regarding the used nomenclature schemes, you can issue [mdciao.nomenclature.references](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.references.html#mdciao.nomenclature.references) to print them out.

In [ ]:
import mdciao

## Download Trajectory Data
Throughout the notebook, we will use the same aliases used by [mdciao.examples.fetch_example_data](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.examples.fetch_example_data.html#mdciao-examples-fetch-example-data) to adress the different sytems/datasets, `"b2ar@Gs", "ghrelin@ghsr" , "mor@muor", "y1_apo"`, but one could create an alias dictionary for nicer tagging of plots etc (see note at the bottom of the notebook).

In [ ]:
systems = ["b2ar@Gs",   
           "ghrelin@ghsr" , 
           "mor@muor", 
           "y1_apo"]
for system in systems:
    d = mdciao.examples.fetch_example_data(system, 
                                           unzip=system,
                                           skip_on_existing=True)

## Nomenclature Data
We will get the nomenclature data, i.e. the per-residue consensus labels mapped to the canonical primary sequence of the receptor, from the [GPCRdb](https://gpcrdb.org/) (in this case). The lookup happens via [UniProt Entry Names](https://www.uniprot.org/help/difference%5Faccession%5Fentryname) and uses mdciao's [nomenclature clases](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/mdciao.nomenclature.html). 

These objects contain all nomenclature information, and map the consensus labels and fragments ("3.50", or "TM3", respectively) not only to the canonical sequence, but to tht of the topologies at hand, using class methods like [top2labels](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.LabelerGPCR.html#mdciao.nomenclature.LabelerGPCR.top2labels) and [top2frags](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.LabelerGPCR.html#mdciao.nomenclature.LabelerGPCR.top2frags). 

So, as a user, you need to know these [UniProt Entry Names](https://www.uniprot.org/help/difference%5Faccession%5Fentryname) for each one of your systems.

In [ ]:
key2GPCR_UniProt = {"b2ar@Gs" :      "adrb2_human", 
                    "ghrelin@ghsr" : "ghsr_human", 
                    "mor@muor" :     "oprm_mouse", 
                    "y1_apo" :       "npy1r_human"
                   }
GPCR = {key : mdciao.nomenclature.LabelerGPCR(val, scheme="BW", 
                                              write_to_disk=True, local_path=key) for key, val in key2GPCR_UniProt.items()}

For the `"b2ar@Gs` system, we also get the [CGN labels](https://www.mrc-lmb.cam.ac.uk/CGN/faq.html), i.e. those for the G-protein.

In [ ]:
CGN = {"b2ar@Gs" : mdciao.nomenclature.LabelerCGN("gnas2_human", write_to_disk=True, local_path="b2ar@Gs")}

Note that in the above cells we're storing the retrieved data as `.xlsx`-files in the individual directories.

## Residue Neighborhood of 3.50
We start by computing the residue neighborhood of notorious residue `3.50` of TM3 in all systems, without even knowing what residue precisely is `3.50` in all systems.

Since we will store the results in the `rn` dictionary and compare them across systems later, we're supressing all outputs in the cell below, but feel free to comment out the `%%capture` and the ` #figures=False` statements

In [ ]:
%%capture
rn = {}
for system in systems:
    rn[system] = list(mdciao.cli.residue_neighborhoods("3.50", f"{system}/traj.xtc", topology=f"{system}/top.pdb", 
                                                       output_dir=system, GPCR_UniProt=GPCR[system], accept_guess=True, 
                                                       ctc_control=1.0,
                                                       no_disk=True, 
                                                       figures=False,
                                                       CGN_UniProt=CGN.get(system,None),
                                                       fragments="chains").values())[0]

## Compare Frequency Bars
We show all contact frequencies of all four systems together using [mdciao.plots.compare_groups_of_contacts](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.plots.compare_groups_of_contacts.html#mdciao-plots-compare-groups-of-contacts). 

To show what the plot would look like without the consensus labels, initially we won't make use of them:

In [ ]:
fig, ax ,plotted_freqs = mdciao.plots.compare_groups_of_contacts(rn, ctc_cutoff_Ang=4, 
                                                                 defrag=None);

This plot is a mess, since all `3.50` residues are different residues in all systems:
* R131 in b2ar@Gs
* R141 in ghrelin@ghsr
* R165 in mor@muor
* R138 in y1_apo

Of course, the same goes for all other residues. This means there's no shared contact pairs to be compared agains each other when using residue names. 

However, if we specifiy ``AA_format="try_consensus"``, the method will try to use those labels (which are unified across systems) when possible:

In [ ]:
fig, ax ,plotted_freqs = mdciao.plots.compare_groups_of_contacts(rn, ctc_cutoff_Ang=4, defrag=None, 
                                                                 AA_format="try_consensus",                                                         
                                                                 anchor="3.50",
                                                                 sort_by="consensus",
                                                                );

Much nicer. We also make the plot even more compact by using:
* `anchor="3.50"` this eliminates "3.50" from all labels and only uses the label of the other residue in the residue pair.
* `sort_by="consensus"` sorts the contacts, insted of by descending frequency (like the first plot), by their consensus label.

Also note, in `b2ar@Gs`, one residue corresponds to the Gs-unit, the 23rd residue of the α5 helix.

## Interface: TM3 vs all other TMs
We now compute whole interfaces between fragments following the same approach, i.e using consensus labels. In this case, we're computing the contacts of the `TM3` with all other elements of the system. We're doing so by using the keyword arguments:

```python
interface_selection_1="TM3",  
interface_selection_2="*", 
```

For the purposes of this notebook, we focus on the usage of consensus descriptors, but please read the full documentation in [mdciao.cli.interface](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.interface.html) for how interfaces can be defined.

Also note that we're supressing the output since we will be comparing (like above) the different systems later. Just comment out the `%%capture` if you want to see the output and the figures.
### Compute

In [ ]:
%%capture
intf = {}
for system in systems:
    intf[system] = mdciao.cli.interface(f"{system}/traj.xtc", topology=f"{system}/top.pdb", output_dir=system, 
                                        GPCR_UniProt=GPCR[system], 
                                        accept_guess=True, 
                                        CGN_UniProt=CGN.get(system, None),
                                        ctc_control=1.0,                              
                                        interface_selection_1="TM3",  
                                        interface_selection_2="*", 
                                        no_disk=True,
                                        fragments="chains",
                                        plot_timedep=False,
                                        figures=True,
                                        self_interface=True,
                                        title=f"interface {system}",
                                        n_nearest=4
                                       )

### Compare Frequency Bars
Using the same method as above, we compare now frequencies, but instead of resolving to each individual pair (there's about 150 TM3 vs all contacts), we aggregate the frequencies to each residue using `per_residue=True`. This loses the per-pair information but makes the plot more compact to begin with.

In [ ]:
mdciao.plots.plots.compare_groups_of_contacts(intf, ctc_cutoff_Ang=4, defrag=None, 
                                              per_residue=True,                                     
                                              AA_format="try_consensus", 
                                              figsize=None,
                                              sort_by="consensus",                                 
                                              lower_cutoff_val=1,
                                              interface=True,
                                             );

There's a lot to unpack here, but we we can immediately see that e.g. `3.28` and  `3.37` behave differently across systems. We'll check them later, but now we pick a representation that tries to be compact without losing pair information.

### Compare Flareplots

In [ ]:
from matplotlib import pyplot as plt
myfig, myax = plt.subplots(2,2, sharex=True, sharey=True, figsize=(70,70), tight_layout=True)
for (system, iintf) , iax in zip(intf.items(), myax.flatten()):
    consensus_maps=[GPCR[system]]
    iCGN = CGN.get(system, None)
    if iCGN is not None:
        consensus_maps.append(iCGN)
    iintf.plot_freqs_as_flareplot(4, fragments="chains", 
                                  scheme="consensus_sparse", 
                                  aura=iintf.frequency_sum_per_residue_idx_dict(4, return_array=True),
                                  panelsize=15,
                                  ax=iax,                                  
                                  consensus_maps=consensus_maps)
    iax.set_title(f"{system}: TM3 interface at 4 Angstrom", fontsize=60)
print()
myfig.tight_layout(h_pad=5)

This representation tries to capture the system's topology, and the individual pairs as well as collective behaviours. We have an [entire notebook]() devoted on how these plots work and how one can fine-tune them. One of the easiest things to spot is the difference in TM3 vs TM6 contacts.



### Coarse-Graining to Consensus Fragments
We can also coarse-grain the frequencies to the fragments, showing a bit the structure and scaffolding role of TM3 in the TM-bundle. All this, without having to directly define the fragments for each system:

In [ ]:
import pandas
CG_freqs = {}
for (system, iintf) , iax in zip(intf.items(), myax.flatten()):
    consensus_maps=[GPCR[system]]
    iCGN = CGN.get(system, None)
    if iCGN is not None:
        consensus_maps.append(iCGN)
    CG_freqs[system] = iintf.frequency_as_contact_matrix_CG (4, fragments=mdciao.fragments.get_fragments(iintf.top, "chains"),
                                                             sparse=True,
                                  consensus_labelers=consensus_maps)["TM3"]
df = pandas.DataFrame(CG_freqs).T
df = df[[key for key in mdciao.nomenclature.nomenclature._GPCR_fragments+mdciao.nomenclature.nomenclature._CGN_fragments if key in df.keys()]]
df.round(1).fillna("")

The table above summarized TM3's behavior, contact-wise, with the other elements. Some things are immediately apparent, like `mor@muor` having less contacts with TM2 and more with TM6

## Residue Neighborhood: Select 3.37 via consensus labels without recomputing anything

We now go back to the residue-level analysis. 

We coould re-compute `3.37`'s neighbordhood the same way we did `3.50`'s initially. However, the individual contacts have already been computed when computing TM3's interface, s.t. we can simply filter the interface for any contacts containing `3.37` (or any other residue) using [ContactGroup.select_by_residues](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup.select_by_residues)

In [ ]:
rn337 = {system : iintf.select_by_residues("3.37") for system, iintf in intf.items()}

### Compare Frequency Bars

In [ ]:
mdciao.plots.compare_groups_of_contacts(rn337, ctc_cutoff_Ang=4, AA_format="try_consensus", anchor="3.37", sort_by="consensus");

As we saw in the interface plot, `3.37` in `ghrelin@ghsr` has many more contacts than the other systems. Now we see why: that the Ghrelin-receptor's `3.37` residue is interacting with TM4 and TM5 more than the rest. 

### Compare Contact-Distances
We will inspect this further, first using [violin plots](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.plots.compare_violins.html#mdciao.plots.compare_violins) in the next cell:

In [ ]:
fig, ax, plotted_labels = mdciao.plots.compare_violins(rn337, 
                                                       ctc_cutoff_Ang=4, defrag=None, 
                                                       AA_format="try_consensus", 
                                                       sort_by="consensus",
                                                       anchor="3.37",
                                                       figsize=None);

Turns out, `4.57, 4.60, 5.42`, and `5.43` weren't even included other `3.37` neighborhoods (`mor@muor, b2ar@Gs, y1_apo`), because they never came in contact with `3.37` (or almost never, according to `min_freq=0.1`, the default value of [mdciao.cli.interface](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.interface.html)). 

Still, we can use mdciao's [mdciao.cli.sites](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.cli.sites.html) to provide an explict list of pairs of residues we want to look at, regardless of them forming a contact or not. 

### Sites
You guessed right, we can specifiy them simply using the consensus labels. We stored them in the cell above in the ``plotted_labels`` variable, so it's very easy to pass them on to the site definitions:

In [ ]:
site_337_def = {"name": "selected 3.77 distances", 
                "pairs" : {"consensus" : [f"3.37-{label}" for label in plotted_labels]}}
site_337_def

Again, feel free to comment in `%%capture` to see all outputs, but we will be comparing the distances two cells below anyways.

In [ ]:
%%capture
site_337 = {}
for system in systems:
    site_337[system] = mdciao.cli.sites(site_337_def,
                                        f"{system}/traj.xtc", topology=f"{system}/top.pdb", output_dir=system, 
                                        GPCR_UniProt=GPCR[system], accept_guess=True, 
                                        no_disk=True,allow_partial_sites=True,
                                        CGN_UniProt=CGN.get(system, None),figures=False,
                                        fragments="chains")["selected 3.77 distances"]

In [ ]:
fig, ax, plotted_labels, repframes = mdciao.plots.compare_violins(site_337, defrag=None, 
                                                                  AA_format="try_consensus",                                         
                                                                  anchor="3.37",
                                                                  sort_by="consensus",
                                                                  representatives=True,
                                                                  ctc_cutoff_Ang=4,
                                       );

We see that the distances for`4.60, 5.42`, and `5.43` for `mor@muor, b2ar@Gs, y1_apo` virtually never cross the 4 Å cutoff.

Also note that by using `representatives=True` we have triggered some things in the violin plots. For each system, we have tried to locate a frame in the trajectory in which the shown residue-residue distances adopt values close to the most likely value (=where the violin is widest). You can read about these representative frames here: [ContactGroup.repframes](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.contacts.ContactGroup.html#mdciao.contacts.ContactGroup.repframes).

These frames are represented as dots inside the individual violins and also as returned geometries in form of `mdtraj` [trajectories](https://www.mdtraj.org/1.9.8.dev0/api/generated/mdtraj.Trajectory.html#mdtraj.Trajectory) (stored in the `repframes` returned value) which we can visualize in 3D. 

## Visualizing Representative Frames

The first thing we note is that geometries won't be aligned, because they're all coming from different simulations. Also, `b2ar@Gs` has the whole G-protein as well.

In [ ]:
import nglview
from matplotlib import colors as mplcolors
colors = mdciao.plots.color_dict_guesser("tab10", repframes.keys())
iwd = nglview.NGLWidget()
for ii, (system, geom) in enumerate(repframes.items()):
    iwd.add_trajectory(geom)#, [[gpcr_idxs]]), title="test")
    iwd.clear_representations(component=ii)
    iwd.add_cartoon(component=ii, color=mplcolors.to_hex(colors[system]), name=system, radius=.1)
iwd

To produce a high quality alignment of the receptor structures, even with low primary-sequence identity, we can arrive at a multiple-sequence-alignment (MSA) via the consensus labels, which act as a proxy for sequence identity. For this, we use `mdciao`'s [AlignerConsensus](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.AlignerConsensus.html#mdciao.nomenclature.AlignerConsensus) class. There's a whole notebook about them [here](https://proteinformatics.uni-leipzig.de/mdciao/notebooks/06.MSA_via_Consensus_Labels.html).

In [ ]:
AC = mdciao.nomenclature.AlignerConsensus(GPCR, tops={key : val.top for key, val in repframes.items()})

Now that we have the [AlignerConsensus](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.AlignerConsensus.html#mdciao.nomenclature.AlignerConsensus) object, we can use the [sequence_match](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.nomenclature.AlignerConsensus.html#mdciao.nomenclature.AlignerConsensus.sequence_match) method to get an overview of how different the primary sequences are:  

In [ ]:
AC.sequence_match()

We can do a lot of things with the `AC` object, e.g. this is the consensus-MSA for TM3

In [ ]:
AC.AAresSeq_match("3.*")

### Superpose Geometries

For the alignment we use the selection "?.\*,-6.\*,8.*" which selects all TMs, except TM6 and H8:


In [ ]:
CA_idxs = AC.CAidxs_match("?.*,-6.*,8.*")

In [ ]:
ref_key = "b2ar@Gs"
for ii, (system, geom) in enumerate(repframes.items()):
    if system!=ref_key:
        geom.superpose(repframes[ref_key], atom_indices=CA_idxs[system], ref_atom_indices=CA_idxs[ref_key])

### Show Geometries

In [ ]:
iwd = nglview.NGLWidget()
for ii, (system, geom) in enumerate(repframes.items()):
    iwd.add_trajectory(geom)
    iwd.clear_representations(component=ii)
    iwd.add_cartoon(component=ii, color=mplcolors.to_hex(colors[system]), name=system, radius=.1)    
iwd

### Show 3.37's neighborhood
Finally, we fine-tune the 3D representation to include the most varying contacts of `3.37`: `4.60, 5.42`, and `5.43`, which we show first as a table across the four systems:

In [ ]:
show=["3.37","4.60", "5.42", "5.43"]
AC.AAresSeq_match(",".join(show))

In [ ]:
iwd = nglview.NGLWidget()
ref_key = "b2ar@Gs"
for ii, (system, geom) in enumerate(repframes.items()):
    iwd.add_trajectory(geom, title=system)
    iwd.clear_representations(component=ii)
    iwd.add_cartoon(component=ii, color=mplcolors.to_hex(colors[system]), name=system, radius=.1)
    AAs = " ".join([GPCR[system].conlab2AA[jj][1:] for jj in show])
    iwd.add_licorice(component=ii, selection=f"({AAs}) and not Hydrogen", radius=.5, color=mplcolors.to_hex(colors[system]))
iwd.gui_style = "ngl"
iwd

From the 3D plot above we can make some observations, most clearly we note that ghrelin@ghsr has the bulikest residue, `Y128@3.50`, which is pointing straight down to a region where TM5 appears to have a bulge, precisely towards the `3.50` position, something the other TM5s don't have. 

## Final Remarks
    
Some final observations

* The point of this notebook isn't to arrive at a particular finding but rather to showcase the utility of streamilining the contact-analysis across diffeent systems using consensus nomenclature.

* We have kept the system names as they are downloaded with [mdciao.examples.fetch_example_data](https://proteinformatics.uni-leipzig.de/mdciao/api/generated/generated/mdciao.examples.fetch_example_data.html#mdciao-examples-fetch-example-data), because they all follow the convention of having a `traj.xtc` and `top.pdb` files, but you can map any topology and trajectory files using aliases and dictionaries:
  
   ```python
   alias = {"b2ar@Gs" :       "adrb2", 
             "ghrelin@ghsr" : "ghsr",
             "mor@muor" :     "muor", 
             "y1_apo" :       "y1"
           }
    #these are just examples of possible other topology filenames.
    top = {"adrb2" :   "adrb2/prot1.pdb",  
           "ghsr" :    "ghrelin/system.pdb", 
           "muor" :    "muor/muor.pdb", 
           "y1"   :    "y1/top.pdb"
          }
    #these are just examples of possible other trajectory filenames.
    trajs = {"adrb2" :   "adrb2/traj.xtc",  
            "ghsr" :     "ghrelin/traj1.xtc", "ghrelin/traj2.xtc",
            "muor" :     "muor/run*.xtc", 
            "y1"   :     "y1/run1.xtc"
          }

   ```

* Althouth the trajectories we have been using are similar in number of frames, they are wildly different in simulated physical length, s.o there isn't really much physical or biological sense in comparing them other than for this demo:

```
  * b2ar@Gs:      280 frames, dt = 10ps, 2.8ns  in total
  * ghrelin@ghsr: 411 frames, dt = 10ns, 41μs   in total
  * mor@muor:     400 frames, dt = 10ns, 40μs   in total
  * y1_apo:       528 frames, dt = 50ns, 26.4ns in total
```
